In [1]:
import os

os.environ["CONFIG_APP_DIR"] = "/app/config"


In [2]:
import sys
import os
import pickle
os.chdir("/app")
from omegaconf import OmegaConf

import json
from etl_pipeline.data_processor_engine.json_engine.json_engine import JsonProcessingEngine
from etl_pipeline.custom.ms.payload_loader import PayloadLoader

/env/ds/anaconda/envs/pipeline/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
from etl_pipeline.config import pipeline_config
cn = pipeline_config.cn

In [4]:
# alert = load_alert()

In [5]:
# payload = load_alert()


# Definition

In [6]:
from pipelines.ms.ms_pipeline import MSPipeline
from etl_pipeline.config import pipeline_config


In [7]:
with open(f'notebooks/sample/wm_address_in_payload_format.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)



payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]


In [8]:
payload = payload_json

In [9]:
from etl_pipeline.config import load_agent_configs, pipeline_config

In [10]:
pipeline_config.reload_pipeline_config()
_ = load_agent_configs()

2022-04-07 19:16:39,679 — main — WARNING — load_agent_config:68 — Field in agent configuration for: ALL_CONCAT_NAMES is not registered field in pipeline.yaml
2022-04-07 19:16:39,683 — main — WARNING — load_agent_config:68 — Field in agent configuration for: ALL_PARTY1_NAME_FULL is not registered field in pipeline.yaml
2022-04-07 19:16:39,688 — main — WARNING — load_agent_config:68 — Field in agent configuration for: ALL_CONCAT_NAMES is not registered field in pipeline.yaml
2022-04-07 19:16:39,693 — main — WARNING — load_agent_config:68 — Field in agent configuration for: ALL_PARTY1_NAME_FULL is not registered field in pipeline.yaml
2022-04-07 19:16:39,698 — main — WARNING — load_agent_config:68 — Field in agent configuration for: ALL_PARTY1_ORGANIZATION_NAME is not registered field in pipeline.yaml
2022-04-07 19:16:39,802 — main — WARNING — load_agent_config:68 — Field in agent configuration for: AP_PARTY_TYPE is not registered field in pipeline.yaml


In [11]:
engine = JsonProcessingEngine(pipeline_config)
pipeline = MSPipeline(engine, config=pipeline_config)

2022-04-07 19:16:39,857 — main — WARNING — load_agent_config:68 — Field in agent configuration for: ALL_CONCAT_NAMES is not registered field in pipeline.yaml
2022-04-07 19:16:39,863 — main — WARNING — load_agent_config:68 — Field in agent configuration for: ALL_PARTY1_NAME_FULL is not registered field in pipeline.yaml
2022-04-07 19:16:39,868 — main — WARNING — load_agent_config:68 — Field in agent configuration for: ALL_CONCAT_NAMES is not registered field in pipeline.yaml
2022-04-07 19:16:39,874 — main — WARNING — load_agent_config:68 — Field in agent configuration for: ALL_PARTY1_NAME_FULL is not registered field in pipeline.yaml
2022-04-07 19:16:39,880 — main — WARNING — load_agent_config:68 — Field in agent configuration for: ALL_PARTY1_ORGANIZATION_NAME is not registered field in pipeline.yaml
2022-04-07 19:16:39,983 — main — WARNING — load_agent_config:68 — Field in agent configuration for: AP_PARTY_TYPE is not registered field in pipeline.yaml


## transform standardized to cleansed

In [12]:
payload = pipeline.transform_standardized_to_cleansed(payload)

ALL_CONNECTED_ACCOUNT_NAMES# Transform standardized to application

In [13]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

> /app/pipelines/ms/ms_pipeline.py(396)set_up_entity_type_match()
    394     def set_up_entity_type_match(self, match):
    395         import pdb; pdb.set_trace()
--> 396         if match["AP_PARTY_TYPE"] == match["WLP_TYPE"]:
    397             match["ENTITY_TYPE_MATCH"] = "Y"
    398         else:

ipdb>  match["AP_PARTY_TYPE"] == match["WLP_TYPE"]
True
ipdb> c
2022-04-07 19:16:49,409 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_country. Ignore for HIT TYPE AGENT
2022-04-07 19:16:49,409 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_job. Ignore for HIT TYPE AGENT
2022-04-07 19:16:49,410 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_other. Ignore for HIT TYPE AGENT
2022-04-07 19:16:49,410 — main.engine — WARNING — merge_to_target_col_from_source_

### payload

In [14]:
len(new_payloads)

2

In [15]:
import pandas as pd
out_payload = pd.DataFrame([match for payload in new_payloads   for match in payload['watchlistParty']['matchRecords']])

out_payload['AP_TRIGGERS'].values[0]

{'Doe': {'ADDRESS1_LINE2': ['Jane Doe AA BBB'],
  'CONCAT_ADDRESS': ['Joe Black & Jane Doe 111 A 11TH ST AAA 1A New Jersey NJ']}}

In [16]:
out_payload[[i for i in out_payload.columns if "entity" in i]]

,entity,entityId,entityTextType,entityType,entityVersion,is_entity_match_agent_ap,is_entity_match_agent_wl,ap_all_is_entity_matchs_aggregated,wl_all_is_entity_matchs_aggregated
0,"{'addresses': [{'address1': '111 STREET 999', ...",908043,UPID,03,20150505194929,I,I,[I],[I]
1,{'addresses': {'address': {'address1': '111 ST...,908044,UPID,03,20150505194923,I,I,[I],[I]


In [17]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        
    for num, match in enumerate(payload['watchlistParty']['matchRecords']):
        assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        print(num, 'match')
        print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
        try:
            print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
        except:
            break


PAYLOAD
122438658
0 match
['04/31/1910', '02/31/1900'] ['MAY 6, 1981']
['UNITED STATES OF AMERICA', 'United States'] ['US']
PAYLOAD
122438659
0 match
['04/31/1910', '02/31/1900'] ['MAY 6, 1981']
['UNITED STATES OF AMERICA', 'United States'] ['Poland', 'PL']


In [18]:
import pickle
with open("tests/shared/parsed_payload.pkl", "wb") as f:
    pickle.dump(new_payloads, f)

In [19]:
with open(f'notebooks/sample/wm_address_in_payload_format_2_input_3_match_records.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)



payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]


In [20]:
payload = payload_json

In [21]:
payload = pipeline.transform_standardized_to_cleansed(payload)

In [22]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

> /app/pipelines/ms/ms_pipeline.py(396)set_up_entity_type_match()
    394     def set_up_entity_type_match(self, match):
    395         import pdb; pdb.set_trace()
--> 396         if match["AP_PARTY_TYPE"] == match["WLP_TYPE"]:
    397             match["ENTITY_TYPE_MATCH"] = "Y"
    398         else:

ipdb> 
ipdb> c
2022-04-07 19:16:52,113 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_country. Ignore for HIT TYPE AGENT
2022-04-07 19:16:52,114 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_job. Ignore for HIT TYPE AGENT
2022-04-07 19:16:52,114 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_other. Ignore for HIT TYPE AGENT
2022-04-07 19:16:52,114 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload nam

In [23]:
import pandas as pd
out_payload = pd.DataFrame([match for payload in new_payloads   for match in payload['watchlistParty']['matchRecords']])

out_payload[[i for i in out_payload.columns  if i.endswith("_aggregated")]]

,ap_all_dobs_aggregated,wl_all_dobs_aggregated,ap_all_documents_aggregated,wl_all_documents_aggregated,ap_all_company_names_aggregated,wl_all_company_names_aggregated,ap_all_names_aggregated,wl_all_names_aggregated,ap_all_employer_names_aggregated,wl_all_employer_names_aggregated,...,wl_all_hit_types_aggregated,ap_all_matched_tokens_tp_markeds_aggregated,wl_all_matched_tokens_tp_markeds_aggregated,ap_all_ap_name_tp_markeds_aggregated,wl_all_ap_name_tp_markeds_aggregated,ap_all_ap_id_tp_markeds_aggregated,wl_all_ap_id_tp_markeds_aggregated,ap_all_is_entity_matchs_aggregated,wl_all_is_entity_matchs_aggregated,all_hit_type_aggregated
0,"[04/31/1910, 02/31/1900]","[MAY 6, 1981]","[022368917, 154421273]",[],"[Eva Pladimirova, Pladimir Vutin]",[Joe Ding],"[Eva Pladimirova, Pladimir Vutin]",[Joe Ding],[],[Joe Ding],...,[],[],[],[],[],[],[],[I],[I],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."
1,"[04/31/1910, 02/31/1900]","[MAY 6, 1981]","[022368917, 154421273]",[],"[Eva Pladimirova, Pladimir Vutin]",[Joe Ding],"[Eva Pladimirova, Pladimir Vutin]",[Joe Ding],[],[Joe Ding],...,[],[],[],[],[],[],[],[I],[I],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."
2,"[04/31/1910, 02/31/1900]","[MAY 6, 1981]","[022368917, 154421273]",[],"[Eva Pladimirova, Pladimir Vutin]",[Joe Dong],"[Eva Pladimirova, Pladimir Vutin]",[Joe Dong],[],[Joe Dong],...,[],[],[],[],[],[],[],[I],[I],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."


In [24]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        
    for num, match in enumerate(payload['watchlistParty']['matchRecords']):
        assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        print(num, 'match')
        print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
        try:
            print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
        except:
            break


PAYLOAD
123
0 match
['04/31/1910', '02/31/1900'] ['MAY 6, 1981']
['UNITED STATES OF AMERICA', 'United States'] ['US']
PAYLOAD
122438659
0 match
['04/31/1910', '02/31/1900'] ['MAY 6, 1981']
['UNITED STATES OF AMERICA', 'United States'] ['US']
PAYLOAD
122438659
0 match
['04/31/1910', '02/31/1900'] ['MAY 6, 1981']
['UNITED STATES OF AMERICA', 'United States'] ['Poland', 'PL']


In [25]:
import pickle
with open("tests/shared/parsed_payload_2_payload.pkl", "wb") as f:
    pickle.dump(new_payloads, f)

In [26]:
new_dict = {}
def return_key(dict_, prefix):
    
    if isinstance(dict_, list):
        for num, i in enumerate(dict_):
            return_key(i, prefix +f"[{num}]")
        return
    if isinstance(dict_, str) or dict_ is None:
        new_dict[prefix] = dict_
        return

    for key in dict_:
        if prefix:
            basic_prefix = prefix + "." + key
        else:
            basic_prefix = key
        return_key(dict_[key], basic_prefix)

In [27]:
with open(f'notebooks/sample/alert_in_payload_format.json', 'w') as file:
    json.dump(new_dict ,file)

In [28]:
payload_json = PayloadLoader().load_payload_from_json(new_dict)

In [29]:
payload_json == payload

False

In [30]:
with open(f'notebooks/sample/alert.json', 'r') as file:
    payload = json.loads(file.read())

In [31]:
payload

{'supplementalInfo': {'alertId': 'SANC-ASM-6434404',
  'uniqueCustId': 'R_US_Active_Address_A00170086018_2018-08-28-18.45.50.012201',
  'busDate': '20211127',
  'datasetName': 'R_US_Active_Address',
  'relatedParties': {'party': [{'id': '0002712291',
     'fields': {'taxId': '154421273',
      'taxIdType': 'SSN',
      'partyResidenceCountry': 'United States',
      'partyName': 'Pladimir Vutin',
      'partyLastName': 'Vutin',
      'partyType': 'Individual',
      'countryOfIncorporation': None,
      'dobDate': '02/31/1900',
      'partyPrimaryCitizenshipCountry': 'United States',
      'partyCountryOfBirth': None,
      'partyFirstName': 'Pladimir',
      'partyMiddleName': None,
      'partyId': '0002712291'}},
    {'id': '0004727489',
     'fields': {'taxId': '022368917',
      'taxIdType': 'SSN',
      'partyResidenceCountry': 'United States',
      'partyName': 'Eva Pladimirova',
      'partyLastName': 'Pladimirova',
      'partyType': 'Individual',
      'countryOfIncorporatio